In [26]:
import random
import itertools
import math
from IPython.display import display
import folium

def distance(point1, point2):
    """Calculate Euclidean distance between two points."""
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def total_distance(route, cities):
    """Calculate total distance of a route."""
    return sum(distance(cities[route[i]], cities[route[i + 1]]) for i in range(len(route) - 1)) + distance(cities[route[-1]], cities[route[0]])

def create_population(size, num_cities):
    """Generate an initial population of random routes."""
    return [random.sample(range(num_cities), num_cities) for _ in range(size)]

def rank_population(population, cities):
    """Rank routes based on total distance."""
    return sorted([(route, total_distance(route, cities)) for route in population], key=lambda x: x[1])

def crossover(parent1, parent2):
    """Perform ordered crossover to produce a child."""
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]
    remaining = [city for city in parent2 if city not in child]
    child = [remaining.pop(0) if x == -1 else x for x in child]
    return child

def mutate(route, mutation_rate=0.1):
    """Swap two cities in a route with a small probability."""
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]
    return route

def genetic_algorithm(cities, population_size=100, generations=500):
    """Run the Genetic Algorithm to find the shortest path."""
    population = create_population(population_size, len(cities))
    
    for _ in range(generations):
        ranked_population = rank_population(population, cities)
        parents = [route for route, _ in ranked_population[:10]]
        offspring = [crossover(random.choice(parents), random.choice(parents)) for _ in range(population_size - 10)]
        mutated_offspring = [mutate(child) for child in offspring]
        population = parents + mutated_offspring
    
    best_route, best_distance = rank_population(population, cities)[0]
    return best_route, best_distance

def main():
    """Main function to get user input and find the shortest route."""
    num_cities = int(input("Enter number of locations: "))
    cities = []
    for i in range(num_cities):
        x, y = map(float, input(f"Enter coordinates of city {i+1} (x y): ").split())
        cities.append((x, y))
    
    best_route, best_distance = genetic_algorithm(cities)
    
    print("\nBest Route (Order of Visiting Cities):", best_route)
    print("Shortest Distance:", round(best_distance, 2))
    
    # Map Representation
    city_map = folium.Map(location=cities[0], zoom_start=12)
    
    for city_index in best_route:
        folium.Marker(location=cities[city_index], popup=f"City {city_index+1}").add_to(city_map)
    
    # Connect cities in the best route
    route_coords = [cities[city_index] for city_index in best_route] + [cities[best_route[0]]]  # Closing the loop
    folium.PolyLine(route_coords, color="blue", weight=2.5, opacity=1).add_to(city_map)
    
    display(city_map)

if __name__ == "__main__":
    main()

Enter number of locations:  2
Enter coordinates of city 1 (x y):  47.6061 22.3328
Enter coordinates of city 2 (x y):  23.0225 72.5714



Best Route (Order of Visiting Cities): [1, 0]
Shortest Distance: 111.86
